# Images

## Introduction to Digital Images

Digital images are stored as a grid (matrix) of colored dots called **pixels**. Each pixel typically has three primary color values:

- **Red**
- **Green**
- **Blue**

Hence, in code, an RGB image can be considered a 3D array (list) with dimensions corresponding to `(height, width, channels)`. For instance, a 640×480 image has 640 pixels in width (horizontal) and 480 in height (vertical), each pixel containing an `(R, G, B)` triplet.

Sometimes, there's an **Alpha** channel (RGBA) representing transparency. We’ll focus on standard RGB for simplicity.

In Python, the **Pillow** library (PIL) is a common tool for reading and writing these images, while libraries like [**diffusers**](https://github.com/huggingface/diffusers) offer utility functions (like `load_image`) for convenience.

We’ll explore both **Pillow**, **numpy** and **diffusers** for:

1. Loading images
2. Manipulating images
3. Creating simple animations (MP4s)

Let’s start by installing and importing the necessary libraries.

We will install all necessary libraries in one step using **pip** like discussed in previous chapters:

`pip install pillow numpy diffusers tqdm`

## Loading Images

There are multiple ways in different libraries to load images stored locally on our device. The most common way is to load it using Pillow (a fort of PIL). See [readthedocs.io](https://pillow.readthedocs.io/en/stable/handbook/tutorial.html) and [automate the boring stuff](https://automatetheboringstuff.com/chapter17/) for more.

In [3]:
from PIL import Image # Import the Image class

img = Image.open("sample.jpg")

We can inspect the loaded image like so:

In [4]:
print('size:', img.size)
print('mode:', img.mode)
print('format:', img.format)

size: (512, 512)
mode: RGB
format: JPEG


To display an image, we can use the show method, which will open the image in the default image-viewer:

In [5]:
img.show()

### Loading from the Web

`Image.open` will only work with local files and return a `PIL`. The `diffusers` library provides a helper function to load images from the web (via ursl) or from disk (using paths).

In [ ]:
from diffusers.utils import load_image

img = load_image("https://thispersondoesnotexist.com/")

The resulting image will be in the `PIL` format aswell.

### Saving images

Images loaded in the `PIL` format can be saved locally with a call to `save` which takes a path as the parameter. If no format is provided it will be chosen according to the file-extension in the path.

In [6]:
from diffusers.utils import load_image

img = load_image("https://thispersondoesnotexist.com/")

img.save("face1.png", format="PNG") # Explicit PNG

img.save("face2.png") # Implicit PNG


## Manipulating Images

Pillow provides many methods for image manipulation. Here are some common ones:

1. **Rotate**: `image.rotate(angle, expand=True)`.
2. **Resize (Scale)**: `image.resize((new_width, new_height))`.
3. **Blur**: using `ImageFilter.BLUR` or other filters from `ImageFilter`.
4. **Enhance Contrast**: using `ImageEnhance.Contrast(image)`.
5. **Composite**: combine multiple images using `Image.composite`.

We'll demonstrate these on an example image.

In [8]:
from diffusers.utils import load_image

img = load_image("https://thispersondoesnotexist.com/")

# 1. Rotate
rotated_image = img.rotate(45, expand=True)
rotated_image.save("img/rotated.png")

![rotated](img/rotated.png)

In [16]:
# 2. Scale
width, height = img.size
half_sized_image = img.resize((width // 2, height // 2))
half_sized_image = img.save("img/scaled.png")

![rotated](img/scaled.png)

In [ ]:
# 3. Blur
from PIL import ImageFilter
blurred_image = img.filter(ImageFilter.BLUR)
blurred_image.save("img/blurred.png")

![rotated](img/blurred.png)

In [19]:
# 4. Enhance Contrast
from PIL import ImageEnhance
enhancer = ImageEnhance.Contrast(img)
contrast_image = enhancer.enhance(2.0)  # Increase contrast
contrast_image.save("img/enhanced.png")

![rotated](img/enhanced.png)

In [20]:
# 5. Composite
# For demonstration, we composite the original and blurred image using a gradient mask.
mask = Image.linear_gradient("L").resize(img.size)
img_rgba = img.convert("RGBA")
blurred_rgba = blurred_image.convert("RGBA")
composited_img = Image.composite(img_rgba, blurred_rgba, mask)
composited_img.save("img/composited.png")

![composited](img/composited.png)

In [22]:
# 6. Crop
cropped = img.crop((0, 0, 256, 50))
cropped.save("img/cropped.png")

![cropped](img/cropped.png)

### Loading Images as numpy

We can also use numpy to read an image direclty as a three-dimensional list of color values. for that we can convert any `PIL` image to `numpy`.

In [23]:
from diffusers.utils import load_image
import numpy as np # numpy is usually renamed to np

# Load as PIL
img_pil = load_image("https://thispersondoesnotexist.com/")


# convert to numpy
img_np = np.asarray(img_pil)
print('dimensions:', img_np.ndim)
print('shape:', img_np.shape)
print('size', img_np.size)

dimensions: 3
shape: (1024, 1024, 3)
size 3145728


In [24]:
print(img_np)

[[[ 77  80  63]
  [ 50  55  33]
  [ 41  50  23]
  ...
  [ 26  39  21]
  [ 23  35  21]
  [ 33  43  32]]

 [[ 52  58  44]
  [ 35  42  26]
  [ 34  41  23]
  ...
  [ 26  39  22]
  [ 26  38  24]
  [ 22  35  25]]

 [[ 39  47  36]
  [ 31  39  28]
  [ 34  42  31]
  ...
  [ 24  35  19]
  [ 21  35  20]
  [ 17  33  20]]

 ...

 [[206 194 182]
  [199 187 173]
  [183 174 157]
  ...
  [197 180 164]
  [199 181 169]
  [207 187 180]]

 [[205 190 187]
  [205 191 182]
  [186 172 161]
  ...
  [196 179 169]
  [211 192 185]
  [206 183 177]]

 [[198 179 183]
  [207 187 186]
  [188 169 162]
  ...
  [203 185 181]
  [219 200 196]
  [201 176 172]]]


We can now manipulate pixel values directly using mathematical functions, but we can also use generative artificial intelligence to write the functions for us. For that we will just need to formulate our prompt accordingly.

As a base prompt we can use the following:

`Give me a python function that takes an image in numpy format (RGB) and manipulates it. The function should return a numpy array in the same format and be called my_function. The function should ...`

Using this prompt and descriptions for what should happen with the image we can manipulate without writing the code ourselves.

For example:

`The function should make the image grayscale.`

`The function should invert the image.`

`The function should sort pixels in the image by brightness.`

Paste the resulting funtion into your code and execute it.

In [25]:
from diffusers.utils import load_image

# Paste your function here:

def my_function(img):
    return img

img_pil = load_image("https://thispersondoesnotexist.com/")
img_np = np.asarray(img_pil)

manipulated_img = my_function(img_np)

Image.fromarray(manipulated_img).save("img/manipulated.png")

## Building a Simple Dynamical System with Image Filters

A **dynamical system** evolves over time by repeatedly applying the same (or similar) rules. Here, we’ll apply filters (blur, sharpen, rotate, etc.) in a loop, treating each new image as the input for the next iteration.

**Learn more**: [Reaction–diffusion systems](https://en.wikipedia.org/wiki/Reaction–diffusion_system)

### Example: Repeated Blur + Sharpen + Rotation
We’ll do the following:
1. Load an initial image.
2. Rotate it slightly, then blur and sharpen.
3. Save each iteration.
4. Export all frames as an MP4 using `export_to_video`.

In [29]:
from diffusers.utils import export_to_video
from tqdm import tqdm

# 1. Define the transformation steps
def blur_then_sharpen(input_image: Image.Image) -> Image.Image:
    blurred = input_image.filter(ImageFilter.BLUR)
    sharpened = blurred.filter(ImageFilter.SHARPEN)
    return sharpened

# 2. Load a base image
original_img = load_image("https://thispersondoesnotexist.com/").convert("RGB")


# 3. Iteratively apply transformations
num_frames = 60
frames = []

current_img = original_img
angle = 0

for i in tqdm(range(num_frames), desc="Generating frames"):
    # Rotate slightly
    rotated = current_img.rotate(angle, expand=True)

    # Blur + Sharpen
    processed = rotated.filter(ImageFilter.BLUR).filter(ImageFilter.SHARPEN)

    # Resize back to original shape if rotation changed dimensions
    processed = processed.resize(original_img.size)

    # Accumulate frames
    frames.append(processed)

    # Prepare next iteration
    current_img = processed
    angle += 1  # 1 degree each iteration

Generating frames: 100%|██████████| 60/60 [00:04<00:00, 13.70it/s]


### Exporting the Frames as an MP4
We can now call `export_to_video` to compile these frames into a video file.

In [37]:
export_to_video(
    video_frames=frames,
    output_video_path="animation.mp4",
    fps=10,
)


It is recommended to use `export_to_video` with `imageio` and `imageio-ffmpeg` as a backend. 
These libraries are not present in your environment. Attempting to use legacy OpenCV backend to export video. 
Support for the OpenCV backend will be deprecated in a future Diffusers version

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\etien\.conda\envs\jupyter-gpu\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\etien\.conda\envs\jupyter-gpu\lib\runpy.py", line 86, in _run_code
    exec(code, ru

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [36]:
!pip install imageio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
